In [61]:
# Dependencies and Setup
import pandas as pd
import statistics as st

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data.head(2)

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411


In [3]:
student_data.head(2)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61


In [4]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(2)
#school_data_complete.count()
#39170 records for each column.  Data appears clean

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635


In [5]:
# list all passing grades for math
pass_math_df = student_data.loc[student_data["math_score"] >= 70,["school_name","math_score","grade"]]
# list all passing grades for reading
pass_reading_df = student_data.loc[student_data["reading_score"] >= 70,["school_name","reading_score","grade"]]

# list all passing grades for both math and reading
pass_all_df = student_data.loc[   (student_data["reading_score"] >= 70) &    
                               (student_data["math_score"] >= 70),
                               ["school_name", "math_score"]]
pass_all_df = pass_all_df.rename(columns={"math_score":"Passed Both"})



In [6]:
district_summary_df = pd.DataFrame({
    'Total Schools': [len(school_data)],
    "Total Students": ['{:,}'.format(len(student_data))],
    "Total Budget": ['{:,}'.format(sum(school_data['budget']))],
    "Average Math Score": [round(st.mean(student_data["math_score"]),2)],
    "Average Reading Score": [round(st.mean(student_data["reading_score"]),2)],
    "% Passing Math": [round(100 * len(pass_math_df) / len(student_data),2)],
    "% Passing Reading": [round(100 * len(pass_reading_df) / len(student_data),2)],
    "% Overall Passing": [round(100 * len(pass_all_df) / len(student_data),2)]
})


###
###
### District Summary:
district_summary_df = district_summary_df.set_index("Total Schools")
print("District Summary:")
district_summary_df.head()

District Summary:


,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Total Schools,,,,,,,
15,"39,170","24,649,428",78.99,81.88,74.98,85.81,65.17


In [7]:
#Start of Schools Summary



'''done:    Need to include:
     x      School name
     x      School type  
     x      Total students
     x      Total school budget
     x      Per student budget
     x      Average math score
     x      Average reading score
     x      % passing math (the percentage of students who passed math)
     x      % passing reading (the percentage of students who passed reading)
     x      % overall passing (the percentage of students who passed math AND reading)'''

schools_total_df = school_data_complete[[
    "school_name","type","Student ID","budget"
]]
schools_total_df.head()

,school_name,type,Student ID,budget
0,Huang High School,District,0,1910635
1,Huang High School,District,1,1910635
2,Huang High School,District,2,1910635
3,Huang High School,District,3,1910635
4,Huang High School,District,4,1910635


In [8]:
#make a data frame that lists high school, type, budget, and count of all students
grouped_schools_df = schools_total_df.groupby(["school_name","type","budget"])
grouped_count_df = grouped_schools_df.count()
grouped_count_df = grouped_count_df.rename(columns={"Student ID":"Student Count"})
grouped_count_df = grouped_count_df.reset_index()
grouped_count_df = grouped_count_df.set_index("school_name")
grouped_count_df.head()

,type,budget,Student Count
school_name,,,
Bailey High School,District,3124928,4976
Cabrera High School,Charter,1081356,1858
Figueroa High School,District,1884411,2949
Ford High School,District,1763916,2739
Griffin High School,Charter,917500,1468


In [9]:
# make data frames to show pass summaries for math, reading, and both
pass_math_group = pass_math_df.groupby(["school_name"])
count_pass_math = pass_math_group.count()

pass_reading_group = pass_reading_df.groupby(["school_name"])
count_pass_reading = pass_reading_group.count()

pass_all_group = pass_all_df.groupby(["school_name"])
count_pass_all = pass_all_group.count()

#merge these DFs together
merge_pass_1 = pd.merge(count_pass_math,  count_pass_reading,  on="school_name", how="left")
#merge_pass_1.head()

merge_pass_2 = pd.merge(merge_pass_1, count_pass_all, on="school_name", how="left")
merge_pass_2 = merge_pass_2.rename(columns={"math_score":"Passed Math", "reading_score":"Passed Reading"})
merge_pass_2.head(2)


,Passed Math,grade_x,Passed Reading,grade_y,Passed Both
school_name,,,,,
Bailey High School,3318,3318,4077,4077,2719
Cabrera High School,1749,1749,1803,1803,1697


In [10]:
#merge passed data frame into school summary DF
merged_school_data = pd.merge(grouped_count_df,merge_pass_2,on="school_name")


In [11]:
merged_school_data["Budget Per Student"]= merged_school_data["budget"] / merged_school_data["Student Count"]
#merged_school_data.head(3)

In [12]:
# groupby school for avg scores
student_score_reduced_df = student_data[["school_name","math_score","reading_score"]]
student_score_group = student_score_reduced_df.groupby(["school_name"])
avg_student_score_per_school = student_score_group.mean().round(2)
#avg_student_score_per_school

#merge into school data
school_data_with_avg_scores = pd.merge(merged_school_data, avg_student_score_per_school, on="school_name")
school_data_with_avg_scores.head()

,type,budget,Student Count,Passed Math,grade_x,Passed Reading,grade_y,Passed Both,Budget Per Student,math_score,reading_score
school_name,,,,,,,,,,,
Bailey High School,District,3124928,4976,3318,3318,4077,4077,2719,628.0,77.05,81.03
Cabrera High School,Charter,1081356,1858,1749,1749,1803,1803,1697,582.0,83.06,83.98
Figueroa High School,District,1884411,2949,1946,1946,2381,2381,1569,639.0,76.71,81.16
Ford High School,District,1763916,2739,1871,1871,2172,2172,1487,644.0,77.10,80.75
Griffin High School,Charter,917500,1468,1371,1371,1426,1426,1330,625.0,83.35,83.82


In [143]:
school_data_with_avg_scores["% Passed Math"] = (100 * school_data_with_avg_scores["Passed Math"] / school_data_with_avg_scores["Student Count"]).round(2)

school_data_with_avg_scores["% Passed Reading"] = (100 * school_data_with_avg_scores["Passed Reading"] / school_data_with_avg_scores["Student Count"]).round(2)

school_data_with_avg_scores["% Passed Both"] = (100 * school_data_with_avg_scores["Passed Both"] / school_data_with_avg_scores["Student Count"]).round(2)

schools_summary = school_data_with_avg_scores[["type",  "Student Count",  "budget",  "Budget Per Student",  "math_score",
    "reading_score",  "% Passed Math",  "% Passed Reading",  "% Passed Both"]]


#school_data_with_avg_scores.head()
#print(schools_summary.columns)
schools_summary = schools_summary.reset_index()
schools_final = schools_summary.rename(columns={
    "school_name":"School",
    "type": "Type",
    "budget": "Total Budget",
    "math_score": "Avg Math Score",
    "reading_score": "Avg Reading Score"
})
schools_final = schools_final.set_index("School")
# Uncomment below to set commas for README.md output
#schools_final["Student Count"] = schools_final["Student Count"].map('{:,d}'.format)
#schools_final["Total Budget"] = schools_final["Total Budget"].map('{:,d}'.format)
###
###
### School Summary:
print("School Summary data:")
schools_final

School Summary data:


,Type,Student Count,Total Budget,Budget Per Student,Avg Math Score,Avg Reading Score,% Passed Math,% Passed Reading,% Passed Both
School,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,53.54


In [67]:
top_five_schools = schools_final.sort_values("% Passed Both", ascending = False).head(5)
print("Top Five Schools:")

top_five_schools

Top Five Schools:


,Type,Student Count,Total Budget,Budget Per Student,Avg Math Score,Avg Reading Score,% Passed Math,% Passed Reading,% Passed Both
School,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,90.54


In [68]:
bottom_five_schools = schools_final.sort_values("% Passed Both").head(5)
print("Bottom Five Schools:")
bottom_five_schools

Bottom Five Schools:


,Type,Student Count,Total Budget,Budget Per Student,Avg Math Score,Avg Reading Score,% Passed Math,% Passed Reading,% Passed Both
School,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,53.54


In [65]:
#Calculate average math score per school for each grade level

#separate scores by grade level
math_score_9 = student_data[['school_name', 'math_score']].loc[student_data['grade']=='9th'].rename(columns={'math_score':'9th'})
math_score_10 = student_data[['school_name', 'math_score']].loc[student_data['grade']=='10th'].rename(columns={'math_score':'10th'})
math_score_11 = student_data[['school_name', 'math_score']].loc[student_data['grade']=='11th'].rename(columns={'math_score':'11th'})
math_score_12 = student_data[['school_name', 'math_score']].loc[student_data['grade']=='12th'].rename(columns={'math_score':'12th'})

#groupby step 1
math_group_9 = math_score_9.groupby(['school_name'])
math_group_10 = math_score_10.groupby(['school_name'])
math_group_11 = math_score_11.groupby(['school_name'])
math_group_12 = math_score_12.groupby(['school_name'])

#groupby step 2
math_avg_9 = math_group_9.mean().round(2)
math_avg_10 = math_group_10.mean().round(2)
math_avg_11 = math_group_11.mean().round(2)
math_avg_12 = math_group_12.mean().round(2)

#merge together
math_avg1 = pd.merge(math_avg_9, math_avg_10, how='left', on = ['school_name'])
math_avg2 = pd.merge(math_avg1, math_avg_11, how='left', on = ['school_name'])
math_avg = pd.merge(math_avg2, math_avg_12, how='left', on = ['school_name'])

# rename index
math_avg = math_avg.reset_index()
math_avg = math_avg.rename(columns={"school_name": "School"})
math_avg = math_avg.set_index("School")

# show result
print("Math Score Average by School and Grade:")
math_avg

Math Score Average by School and Grade:


,9th,10th,11th,12th
School,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [64]:
#Calculate average reading score per school for each grade level

#separate scores by grade level
reading_score_9 = student_data[['school_name', 'reading_score']].loc[student_data['grade']=='9th'].rename(columns={'reading_score':'9th'})
reading_score_10 = student_data[['school_name', 'reading_score']].loc[student_data['grade']=='10th'].rename(columns={'reading_score':'10th'})
reading_score_11 = student_data[['school_name', 'reading_score']].loc[student_data['grade']=='11th'].rename(columns={'reading_score':'11th'})
reading_score_12 = student_data[['school_name', 'reading_score']].loc[student_data['grade']=='12th'].rename(columns={'reading_score':'12th'})

#groupby step 1
reading_group_9 = reading_score_9.groupby(['school_name'])
reading_group_10 = reading_score_10.groupby(['school_name'])
reading_group_11 = reading_score_11.groupby(['school_name'])
reading_group_12 = reading_score_12.groupby(['school_name'])

#groupby step 2
reading_avg_9 = reading_group_9.mean().round(2)
reading_avg_10 = reading_group_10.mean().round(2)
reading_avg_11 = reading_group_11.mean().round(2)
reading_avg_12 = reading_group_12.mean().round(2)

#merge together
reading_avg1 = pd.merge(reading_avg_9, reading_avg_10, how='left', on = ['school_name'])
reading_avg2 = pd.merge(reading_avg1, reading_avg_11, how='left', on = ['school_name'])
reading_avg = pd.merge(reading_avg2, reading_avg_12, how='left', on = ['school_name'])

# rename index
reading_avg = reading_avg.reset_index()
reading_avg = reading_avg.rename(columns={"school_name": "School"})
reading_avg = reading_avg.set_index("School")

# show result
print("Reading Score Average by School and Grade:")
reading_avg

Reading Score Average by School and Grade:


,9th,10th,11th,12th
School,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [147]:
budget_bins = [575,600,625,650,675]
budget_names = ["575-600","600-625","625-650","650-675"]


schools_final["Budget Range"] = pd.cut(schools_final["Budget Per Student"], budget_bins, labels=budget_names, include_lowest=True)
budget_df = schools_final[[
    "Avg Math Score","Avg Reading Score", "% Passed Math","% Passed Reading","% Passed Both", "Budget Range"
]]
budget_group_df = budget_df.groupby("Budget Range")
budget_summary_df = budget_group_df.mean().round(2)

print("Average Scores by Budget Range:")
budget_summary_df

Average Scores by Budget Range:


,Avg Math Score,Avg Reading Score,% Passed Math,% Passed Reading,% Passed Both
Budget Range,,,,,
575-600,83.43,83.89,93.54,96.46,90.27
600-625,83.60,83.93,93.99,96.54,90.57
625-650,78.03,81.42,71.11,83.45,59.94
650-675,76.96,81.06,66.22,81.09,53.52


In [141]:
size_bins = [0,1799,2925,5000]
size_names = ["small","medium","large"]
size_bins_df = schools_final.reset_index()
size_bins_df["Student Count Ranges"] = pd.cut(size_bins_df["Student Count"], size_bins, labels=size_names, include_lowest=True)

size_group = size_bins_df.groupby(["Student Count Ranges"])
size_range_df = size_group[['Avg Math Score','Avg Reading Score','% Passed Math','% Passed Reading','% Passed Both']].mean().round(2)

print("Grades by Student Count Ranges:")
size_range_df

Grades by Student Count Ranges:


,Avg Math Score,Avg Reading Score,% Passed Math,% Passed Reading,% Passed Both
Student Count Ranges,,,,,
small,83.55,83.85,93.53,96.50,90.24
medium,80.75,82.77,83.06,90.16,76.01
large,76.99,80.97,66.37,80.99,53.58


In [156]:
type_df = schools_final.reset_index()
type_reduced_df = type_df[["Type",'Avg Math Score','Avg Reading Score','% Passed Math',
                         '% Passed Reading','% Passed Both']]
type_group = type_reduced_df.groupby(["Type"])
type_summary_df = type_group.mean().round(2)

print("Grades by School Type:")
type_summary_df

Grades by School Type:


,Avg Math Score,Avg Reading Score,% Passed Math,% Passed Reading,% Passed Both
Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
